First we import the library. 

In [1]:
import sys
sys.path.append('../build')
import stlrom

We instantiate an STLDriver object. This object is the main interface to the library. 

In [2]:
monitor = stlrom.STLDriver()

We need to declare signals, parameters (optional), and formulas. For this we can use the `parse_string()` method. 

In [3]:
monitor.parse_string("signal x, y")       # declaring signals named 'x' and 'y'
monitor.parse_string("param p=0.2, tau=5") # declaring parameters named 'p' and 'q'

True

Let's declare formulas. 

In [4]:
monitor.parse_string("mu_x := x[t] > 0 ") # a atomic proposition
monitor.parse_string("mu_y := y[t] < p ") # another atomic proposition
monitor.parse_string("mu := mu_x and mu_y") # a compound proposition
monitor.parse_string("phi := ev_[0, tau] mu") # a temporal proposition
monitor.disp()


Assigned formulas:
-------------------
mu:
x[t] > 0 and y[t] < p
mu_x:
x[t] > 0
mu_y:
y[t] < p
phi:
ev_[0,tau] (x[t] > 0 and y[t] < p) 

Default Parameters:
---------------------
p=0.2
tau=5



We can now add data. We do it by adding time valued samples of the form [t, x, y]. Be careful to respect the order of signals declaration.

In [5]:
monitor.add_sample([0, 2., 3.])     # at time 0, x=2, y=3
monitor.add_sample([0.5, -1., -4.]) # at time 0.5, x=-1, y=-4
monitor.add_sample([3., 3., -5.])    # etc.     
print(monitor.data)

[[0.0, 2.0, 3.0], [0.5, -1.0, -4.0], [3.0, 3.0, -5.0]]


We are now ready to evaluate formulas. For this we use the get_online_rob() method. 

In [6]:
rob_mu_x  = monitor.get_online_rob("mu_x")
rob_mu_y  = monitor.get_online_rob("mu_y")
rob_mu    = monitor.get_online_rob("mu")
rob_phi = monitor.get_online_rob("phi")

print("rob_mu_x = ", rob_mu_x)
print("rob_mu_y = ", rob_mu_y)
print("rob_mu = ", rob_mu)
print("rob_phi = ", rob_phi)

rob_mu_x =  [2.0, 2.0, 2.0]
rob_mu_y =  [-2.8, -2.8, -2.8]
rob_mu =  [-2.8, -2.8, -2.8]
rob_phi =  [3.0, 3.0, 10000.0]


`get_online_rob("phi")` returns three values: `[rob_estimate, rob_low, rob_up]`. They have the following characteristics:
- if the `phi` can be fully evaluated by the data, then all three values are equal to the actual robust satisfaction `rob` of the formula. On the example above, this is the case for atomic propositions, which are evaluated at time 0 by default.
- Otherwise, `[rob_estimate, rob_low, rob_up]` are respectively an estimate, a lower bound and an upper bound of `rob`. In the example above, `phi` cannot be evaluated because we have data up to time 3 and the formula requires data up to time `tau`, which is 5. However, stlrom computed that rob is between 3 and 10000, which means that we can already assert that the formula is satisfied (rob is positive.) 

Note that 10000 is an arbitrary large number that can be changed using the set_BigM() method.

In [7]:
s = stlrom.Signal().set_BigM(5000) # set the BigM value globally to 5000

In [8]:
rob_phi = monitor.get_online_rob("phi")
print(rob_phi)

[3.0, 3.0, 5000.0]


Parameters can be changed using set_param().


In [9]:
monitor.set_param("tau", 3.) # change the value of the parameter tau, from 5 to 3
rob_phi = monitor.get_online_rob("phi")
print(rob_phi) 

[3.0, 3.0, 3.0]


Note that the robustness value of phi is now fully computed, since we reduced the time bound of the eventually operator. get_online_rob() can be called at a time different than 0.

In [10]:
rob_mu_at_1 = monitor.get_online_rob("mu_x", 1) 
print("rob_mu_at_1:", rob_mu_at_1)

rob_phi_at_1 = monitor.get_online_rob("phi", 1) 
print("rob_phi_at_1:", rob_phi_at_1)


rob_mu_at_1: [-0.19999999999999996, -0.19999999999999996, -0.19999999999999996]
rob_phi_at_1: [3.0, 3.0, 5000.0]


Note that data is interpolated linearly between samples. Also from time 1, the horizon of the eventually operator again goes beyond what we have in the data.